In [0]:
# Notebook 1: BRONZE - Carga incremental

from pyspark.sql import SparkSession
import requests
import json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, MapType

spark = SparkSession.builder.getOrCreate()

schema = StructType([
    StructField("id", LongType(), True),
    StructField("name", StringType(), True),
    StructField("symbol", StringType(), True),
    StructField("slug", StringType(), True),
    StructField("cmc_rank", LongType(), True),
    StructField("last_updated", StringType(), True),
    StructField("quote", MapType(StringType(), StructType([
        StructField("price", DoubleType(), True),
        StructField("market_cap", DoubleType(), True),
        StructField("volume_24h", DoubleType(), True),
        StructField("percent_change_1h", DoubleType(), True),
        StructField("percent_change_24h", DoubleType(), True),
        StructField("percent_change_7d", DoubleType(), True),
        StructField("last_updated", StringType(), True)
    ])))
])

url = "https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
parameters = {"start": "1", "limit": "100", "convert": "USD"}
headers = {
    "Accepts": "application/json",
    "X-CMC_PRO_API_KEY": "e734d13f-5b0b-481b-8490-9f7d35c66976",
}

response = requests.get(url, headers=headers, params=parameters)
data = response.json()
cryptos = data["data"]

df_bronze = spark.createDataFrame(cryptos, schema=schema)

BRONZE_TABLE = "workspace.javier_altoe.crypto_bronze"

# En lugar de overwrite, usar append
df_bronze.write.format("delta").mode("append").saveAsTable(BRONZE_TABLE)

print("✅ Datos añadidos a Bronze:", BRONZE_TABLE)
